In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

from google.colab import files
files.upload()

import tensorflow as tf
import numpy as np
import csv
from os import getcwd
import csv
from tensorflow.keras.preprocessing.image import ImageDataGenerator

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


Saving kaggle.json to kaggle.json


In [2]:
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!ls ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json 


kaggle.json


In [3]:
!kaggle datasets download -d datamunge/sign-language-mnist -p /content/gdrive/My\ Drive/kaggle/MNIST


sign-language-mnist.zip: Skipping, found more recently modified local copy (use --force to force download)


In [4]:
import os
os.chdir('/content/gdrive/My Drive/kaggle/MNIST')  
!mkdir train  
!mkdir test 
!unzip -q sign-language-mnist.zip -d sign-language-mnist/


mkdir: cannot create directory ‘train’: File exists
mkdir: cannot create directory ‘test’: File exists
replace sign-language-mnist/amer_sign2.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [15]:
def data(file_name):
  with open(file_name) as training_files:
    csv_reader = csv.reader(training_files, delimiter=',')
    first_line = True
    temp_images=[]
    temp_labels = []
    for row in csv_reader:
      if first_line:
        first_line = False
      else:
        temp_labels.append(row[0])
        image_data = row[1:785]
        image_data_as_array = np.array_split(image_data,28)
        temp_images.append(image_data_as_array)


    images = np.array(temp_images).astype('float')
    labels = np.array(temp_labels).astype('float')

    return images, labels


path_mnist_train = f"{getcwd()}/sign-language-mnist/sign_mnist_train/sign_mnist_train.csv"
path_mnist_test = f"{getcwd()}/sign-language-mnist/sign_mnist_test/sign_mnist_test.csv"
training_images, training_labels = data(path_mnist_train)
testing_images, testing_labels = data(path_mnist_test)

print(training_images.shape)
print(training_labels.shape)
print(testing_images.shape)
print(testing_labels.shape)

(27455, 28, 28)
(27455,)
(7172, 28, 28)
(7172,)


In [16]:
training_images = np.expand_dims(training_images,axis = 3)
testing_images = np.expand_dims(testing_images,axis = 3)

Data_generator = ImageDataGenerator(
    rescale = 1./255,
    rotation_range = 5,
    width_shift_range = 0.05,
    height_shift_range  =0.05,
    shear_range = 0.05,
    zoom_range = 0.05,
    horizontal_flip = True,
    fill_mode = 'nearest'
)

validation_data_generator = ImageDataGenerator(rescale = 1./255)

print(training_images.shape)
print(testing_images.shape)

(27455, 28, 28, 1)
(7172, 28, 28, 1)


In [32]:

model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(400,(5,5),activation='relu',input_shape=(28,28,1), padding='same'),
       
        tf.keras.layers.MaxPooling2D((2,2),padding='same'),
        
        tf.keras.layers.Conv2D(200,(4,4),activation='relu',padding='same'),
        tf.keras.layers.Dropout(0.1),
        tf.keras.layers.MaxPooling2D((2,2),padding='same'),
        
        tf.keras.layers.Conv2D(100,(3,3),activation='relu',padding='same'),
       
        tf.keras.layers.MaxPooling2D((2,2),padding='same'),

        tf.keras.layers.Conv2D(50,(2,2),activation='relu',padding='same'),
      
        tf.keras.layers.MaxPooling2D((2,2),padding='same'),

        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(1024,activation = 'relu'),
        tf.keras.layers.Dropout(0.6),
        tf.keras.layers.Dense(512,activation='relu'),
        tf.keras.layers.Dropout(0.7),
        tf.keras.layers.Dense(25,activation='softmax')
])

model.compile(loss = 'sparse_categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
model.summary()

history = model.fit_generator(Data_generator.flow(training_images, training_labels, batch_size=128),
                              steps_per_epoch = len(training_images) / 128,
                              epochs = 18,
                              validation_data=validation_data_generator.flow(testing_images, testing_labels, batch_size=64),
                              validation_steps = len(testing_images) / 64)

model.evaluate(testing_images,testing_labels)



Model: "sequential_21"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_84 (Conv2D)           (None, 28, 28, 400)       10400     
_________________________________________________________________
max_pooling2d_84 (MaxPooling (None, 14, 14, 400)       0         
_________________________________________________________________
conv2d_85 (Conv2D)           (None, 14, 14, 200)       1280200   
_________________________________________________________________
dropout_62 (Dropout)         (None, 14, 14, 200)       0         
_________________________________________________________________
max_pooling2d_85 (MaxPooling (None, 7, 7, 200)         0         
_________________________________________________________________
conv2d_86 (Conv2D)           (None, 7, 7, 100)         180100    
_________________________________________________________________
max_pooling2d_86 (MaxPooling (None, 4, 4, 100)       

[8.125615119934082, 0.9912158250808716]